In [2]:
import numpy as np

def train_threshold_function(T):
    T.sort(key=lambda x: x[0])
    best_threshold = None
    min_misclassification = len(T)
    
    for i in range(len(T) - 1):
        threshold = (T[i][0] + T[i + 1][0]) / 2
        misclassification = sum([1 for x, y in T if (x >= threshold) != y])
        
        if misclassification < min_misclassification:
            min_misclassification = misclassification
            best_threshold = threshold
            
    def classify(x):
        return int(x >= best_threshold)
    return classify

In [3]:

def train_multidimension_threshold_function(T):
    T.sort(key=lambda x: min(x[0]))
    T_projections = [min(x) for x,_ in T]
    best_threshold = None
    min_misclassification = len(T)
    
    for i in range(len(T) - 1):
        threshold = (T_projections[i] + T_projections[i + 1]) / 2
        
        misclassification = sum([1 for x,y in T if int(all(x >= threshold)) != y])
        
        if misclassification < min_misclassification:
            min_misclassification = misclassification
            best_threshold = threshold
    
    def classify(x):
        return int(all(np.array(x) >= best_threshold))
    return classify

In [4]:
def train_multiclass_threshold_function(T):
    # number of classes = number of dimensions according to the problem statement
    thresholds = [0]*len(T[0][0])
    
    for i in range(len(T[0][0])):
        thresholds[i] = train_multidimension_threshold_function([(x, y[i]) for x, y in T])

    def classify(x):
        return [classify(x) for classify in thresholds]
    return classify

In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('iris.data', header=None)
data.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
data['class no.'] = data['species'].astype('category').cat.codes
data.drop(['sepal_length'], inplace=True, axis=1)

def vectorize(x):
    y = [0]*3
    y[x] = 1
    return y

X = data.values[:, :3]
Y = data['class no.'].apply(vectorize)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
data

,sepal_width,petal_length,petal_width,species,class no.
0,3.5,1.4,0.2,Iris-setosa,0
1,3.0,1.4,0.2,Iris-setosa,0
2,3.2,1.3,0.2,Iris-setosa,0
3,3.1,1.5,0.2,Iris-setosa,0
4,3.6,1.4,0.2,Iris-setosa,0
...,...,...,...,...,...
145,3.0,5.2,2.3,Iris-virginica,2
146,2.5,5.0,1.9,Iris-virginica,2
147,3.0,5.2,2.0,Iris-virginica,2
148,3.4,5.4,2.3,Iris-virginica,2


In [64]:
classify = train_multiclass_threshold_function(list(zip(X_train,Y_train)))
data['predicted class'] = [classify(x) for x in X]

data['all correct'] = [int(x == y) for x, y in zip(data['predicted class'],Y)]
data['somewhat correct'] = [int(x[y]) for x,y in zip(data['predicted class'],data['class no.'])]

Y_train_pred = [classify(x) for x in X_train]
train_all_correct = [int(x == y) for x, y in zip(Y_train_pred,Y_train)]
train_somewhat_correct = [int(x[y.index(1)]) for x,y in zip(Y_train_pred,Y_train)]

Y_test_pred = [classify(x) for x in X_test]
test_all_correct = [int(x == y) for x, y in zip(Y_test_pred,Y_test)]
test_somewhat_correct = [int(x[y.index(1)]) for x,y in zip(Y_test_pred,Y_test)]

print(f"{sum(train_all_correct)} all correct classifications out of {len(X_train)} samples of training data: {round(sum(train_all_correct)*100/len(X_train),2)}%")
print(f"{sum(train_somewhat_correct)} somewhat correct classifications out of {len(X_train)} samples of training data: {round(sum(train_somewhat_correct)*100/len(X_train),2)}%")
print(f"{sum(test_all_correct)} all correct classifications out of {len(X_test)} samples of validation data: {round(sum(test_all_correct)*100/len(X_test),2)}%")
print(f"{sum(test_somewhat_correct)} somewhat correct classifications out of {len(X_test)} samples of validation data: {round(sum(test_somewhat_correct)*100/len(X_test),2)}%")
data

42 all correct classifications out of 120 samples of training data: 35.0%
78 somewhat correct classifications out of 120 samples of training data: 65.0%
6 all correct classifications out of 30 samples of validation data: 20.0%
18 somewhat correct classifications out of 30 samples of validation data: 60.0%


,sepal_width,petal_length,petal_width,species,class no.,predicted class,all correct,somewhat correct
0,3.5,1.4,0.2,Iris-setosa,0,"[0, 0, 0]",0,0
1,3.0,1.4,0.2,Iris-setosa,0,"[0, 0, 0]",0,0
2,3.2,1.3,0.2,Iris-setosa,0,"[0, 0, 0]",0,0
3,3.1,1.5,0.2,Iris-setosa,0,"[0, 0, 0]",0,0
4,3.6,1.4,0.2,Iris-setosa,0,"[0, 0, 0]",0,0
...,...,...,...,...,...,...,...,...
145,3.0,5.2,2.3,Iris-virginica,2,"[0, 1, 1]",0,1
146,2.5,5.0,1.9,Iris-virginica,2,"[0, 1, 1]",0,1
147,3.0,5.2,2.0,Iris-virginica,2,"[0, 1, 1]",0,1
148,3.4,5.4,2.3,Iris-virginica,2,"[0, 1, 1]",0,1


We trained the model on 80% of the data set and tested its performance on the rest of the data it had yet to see. We saw great results on validation data that were about equal to the model's performance on training data. This is a good sign that the model is not overfitting. The metrics we used to evaluate our model were "all correct %" and "somewhat correct %". All correct classifications were ones where the correct class was classified as positive and all other classes were classified as negative. Somewhat correct classifications were ones where the correct class was classified as positive